# Meteorological time-series extraction: Part A

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is part of the EStreams publication and was used to extract and aggregate the meteorological time-series from the E-OBS dataset. At Part A we compute the Potential Evapotranspiration using the Hargreaves formulation.

* Note that this code enables not only the replicability of the current database but also the extrapolation to new catchment areas. 
* Additionally, the user should download and insert the original raw-data in the folder of the same name prior to run this code. 
* The original third-party data used were not made available in this repository due to redistribution and storage-space reasons.  

## Requirements
**Python:**

* Python>=3.6
* Jupyter
* os
* pyet=1.2.2
* xarray


Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**

* meteorology/eobs/{rr, tg, tn, tx, pp, hu, fg, qq, elev}_ens_mean_0.25deg_reg_v27.0e.nc   https://www.ecad.eu/download/ensembles/download.php (Last access: 27 November 2023)

**Directory:**

* Clone the GitHub directory locally
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References

* Cornes, R., G. van der Schrier, E.J.M. van den Besselaar, and P.D. Jones. 2018: An Ensemble Version of the E-OBS Temperature and Precipitation Datasets, J. Geophys. Res. Atmos., 123. doi:10.1029/2017JD028200
* Vremec, M. & Collenteur, R. PyEt-a Python package to estimate potential and reference evapotranspiration 1.1.0. in EGU General Assembly Conference Abstracts (2021). https://github.com/pyet-org/pyet

## Licenses
* EOBS: "The ECA&D data policy applies. These observational data are strictly for use in non-commercial research and non-commercial education projects only. Scientific results based on these data must be submitted for publication in the open literature without any delay linked to commercial objectives" https://www.ecad.eu/download/ensembles/download.php#guidance (Last access: 27 November 2023)

## Observations
#### E-OBS filenames

* rr = Total daily precipitation
* tg = Mean daily temperature
* tn = Minimum daily temperature
* tx = Maximum daily temperature
* pp = Mean daily air pressure at sea level
* hu = Mean daily relative humidity
* fg = Mean wind speed at 10-meters
* qq = Total daily global radiation
* elev = Elevation

# Import modules

In [ ]:
import numpy as np
import xarray as xr
import os
import pyet

# Configurations

In [ ]:
# Only editable variables:
# Relative path to your local directory
PATH = "../../.."

# Set the area o crop:
# Basically it is easier if we crop Iceland for the PET computation. Hence we divide the PET results in two. 
# not_iceland and only_iceland. 
CHOSEN_CLIP = "only_iceland"  # Area to be processed ["not_iceland", "only_iceland"]

* #### The users should NOT change anything in the code below here.

In [ ]:
# Non-editable variables:
PATH_netcdfs = "data/meteorology/eobs/"
PATH_OUTPUT = "data/meteorology/eobs/"

# Set the configurations for the clip:
if CHOSEN_CLIP == "not_iceland":
    # For Europe and without Iceland
    lat_min = 34.75
    lat_max = 70.6
    lon_min = -10.5
    lon_max = 45.0
    FILENAME_PET = "pet_hargreaves_025deg_v280e.nc"
else:
    # For Iceland:
    lat_min = 63.0
    lat_max = 67.0
    lon_min = -24.5
    lon_max = -10.5
    FILENAME_PET = "pet_hargreaves_iceland_025deg_v280e.nc"

# Set the directory:
os.chdir(PATH)

# Import data
## Netcdf-files

In [ ]:
# Mean temperature:
tmean = xr.open_dataset(PATH_netcdfs+"tg_ens_mean_0.25deg_reg_v28.0e.nc", 
                        engine="netcdf4").sel(longitude=slice(lon_min, lon_max), 
                                              latitude=slice(lat_min, lat_max))["tg"]
# Maximum temperature:                                          
tmax = xr.open_dataset(PATH_netcdfs+"tx_ens_mean_0.25deg_reg_v28.0e.nc", 
                       engine="netcdf4").sel(longitude=slice(tmean.longitude.min(), tmean.longitude.max()), 
                                             latitude=slice(tmean.latitude.min(), tmean.latitude.max()))["tx"]
# Minimum temperature:                                            
tmin = xr.open_dataset(PATH_netcdfs+"tn_ens_mean_0.25deg_reg_v28.0e.nc", 
                       engine="netcdf4").sel(longitude=slice(tmean.longitude.min(), tmean.longitude.max()), 
                                             latitude=slice(tmean.latitude.min(), tmean.latitude.max()))["tn"]
# Elevation:
elevation =  xr.open_dataset(PATH_netcdfs+"elev_ens_0.25deg_reg_v28.0e.nc", 
                     engine="netcdf4").sel(longitude=slice(tmean.longitude.min(), tmean.longitude.max()), 
                                           latitude=slice(tmean.latitude.min(), tmean.latitude.max()))["elevation"].fillna(0)

# Processing

In [ ]:
# Create the lat file:
lat = tmean.latitude * np.pi / 180
lat = lat.expand_dims(dim={"longitude":tmean.longitude}, axis=1)

# Process with the computation:
hargreaves = pyet.hargreaves(tmean, tmax, tmin, lat=lat)

## Data export

In [ ]:
hargreaves.to_netcdf(PATH_OUTPUT+FILENAME_PET)

# End